In [1]:
#Imports
import pandas as pd

# En este fichero se preparan los datos para posteriormente hacer correlaciones lineales con ellos en otros archivos .ipynb

Qué hay en el documento:

1) Obtener los datos cada 15 mins de policromos

2) Separar datos por semanas


# 0.1) Obtener los datos cada 15 mins de policromos:
Los datos de temperaturas y CO2  se toman cada minuto en la Sala de Policromos, en contraste con el resto de zonas de la cueva, donde los datos se toman cada 15 minutos. Para poder buscar correlaciones entre los datos de Policromos y de otras salas es necesario obtener los datos de Policromos con la misma periodicidad que otras salas.

In [2]:
def adaptar_len_policromos(FicheroCruce, FicheroExterior, FicheroHall, FicheroPolicromos, mesAño):
    """
    Atributos:
    1) FicheroCruce: Excel con datos del Cruce
    2) FicheroExterior: Excel con datos del Exterior
    3) FicheroHall: Excel con datos del Hall
    4) FicheroPolicromos: Excel con datos de Policromos
    5) mesAño: String que indique el mes y año al que pertenecen los datos 
    
    Devuelve:
    datosPolicromos = Tp, Tp_suavizada, CO2, CO2_suavizado, Fecha_Tiempo; Una lista de listas, donde cada elemento (Tp, 
    Tp_suavizada, ect.) contiene los datos cada 15 minutos de policromos de la variable correspondiente.
    
    Contenido de datosPolicromos:
    - Tp: Temperatura medida en Policromos en grados Celsius
    - Tp_suavizada:  Temperatura medida en Policromos en grados Celsius, trás aplicarle el suavizado de datos
    - CO2: Contenido de CO2 en partes por millón (ppm) medido en Policromos
    - CO2_suavizado: Contenido de CO2 en partes por millón (ppm) medido en Policromos trás realizar el suavizado de datos
    - Fecha_Tiempo: timestamp con la fecha y la hora exacta a la que se tomaron las medidas
    
    Cosas a tener en cuenta:
    Esta función pinta un mensaje de aviso en caso de que la longitud de policromos sea diferente a la del resto de ficheros;
    Esto solo ocurrirá en caso de que el archivo Policromos sea más corto que los demás, en caso de que se haya estropeado algún 
    sensor o haya habido algún problema en la obtención de los datos.
    
    La función pinta la longitud de los datos una vez realizados los cambios en Policromos, junto con el dato inicial y final de
    'Fecha/Tiempo' para las cuatro salas.
    """
    print('---------------------------------------------------------------------------------------------------------------')
    print('---------------------------------------------------------------------------------------------------------------')
    print(mesAño)
    #Leer ficheros
    Cruce = pd.read_excel(FicheroCruce)
    Exterior = pd.read_excel(FicheroExterior)
    FicheroHall = pd.read_excel(FicheroHall)
    Policromos = pd.read_excel(FicheroPolicromos)
    
    #Copiar los datos cada 15 mins
    a=0
    b=len(Policromos['Fecha/Tiempo'])
    min1 =FicheroHall['Fecha/Tiempo'][0].minute #Minutos del 1er dato del fichero de referencia
    Tp=[]#El 0 es un número entero, coge desde el 1er dato útil
    Tp_suavizada=[]
    CO2=[]
    CO2_suavizado=[]
    Fecha_Tiempo = []
    while a<b-1:
        division=(Policromos['Fecha/Tiempo'][a].minute-min1)/15
        division_entera=(Policromos['Fecha/Tiempo'][a].minute-min1)//15
        if  division == division_entera and division>=0:
            Tp.append(Policromos['Temperatura C (C)'][a])
            Tp_suavizada.append(Policromos['Tp_suavizada'][a])
            CO2.append(Policromos['CO2 C (ppm)'][a])
            CO2_suavizado.append(Policromos['CO2p_suavizado'][a])
            Fecha_Tiempo.append(Policromos['Fecha/Tiempo'][a])
        a=a+1
    # Tenemos listas de Tp, Tp_suavizada, CO2, CO2_suavizado y Fecha/Tiempo cada 15 mins 
    
    #recortar ficheros en caso de que la longitud de los elemento de datosPolicromos sea mayor que la del resto de salas:
    if len(FicheroHall['Fecha/Tiempo'])< len(Tp):
        Tp=Tp[:len(FicheroHall['Fecha/Tiempo'])]
        Tp_suavizada=Tp_suavizada[:len(FicheroHall['Fecha/Tiempo'])]
        CO2=CO2[:len(FicheroHall['Fecha/Tiempo'])]
        CO2_suavizado=CO2_suavizado[:len(FicheroHall['Fecha/Tiempo'])]
        Fecha_Tiempo=Fecha_Tiempo[:len(FicheroHall['Fecha/Tiempo'])]
    elif len(FicheroHall['Fecha/Tiempo'])> len(Tp):
        print('Policromos es más corto-> Solucionar')  
    # No hace falta hace un elif adicional con la condición 'len(FicheroHall['Fecha/Tiempo'])= len(Tp)'. No da problemas
    
    #Pintar datos para comprobar que las longitudes de todos los archivos sean iguales, y que empiezan y terminan a la vez:
    print('Datos finales-------------------')
    print('lenCruce:', 'lenExterior:', 'lenHall:', 'lenPolicromos:')
    print(len(Cruce['Fecha/Tiempo']), len(Exterior['Fecha/Tiempo']), len(FicheroHall['Fecha/Tiempo']), len(Tp))
    print('--------------------------------')
    print('1er dato Cruce:', '1er dato Exterior:', '1er dato hall:', '1er dato Policromos:')
    print(Cruce['Fecha/Tiempo'][0], Exterior['Fecha/Tiempo'][0],FicheroHall['Fecha/Tiempo'][0],Fecha_Tiempo[0])
    print('--------------------------------')
    print('Último dato Cruce:', 'Último dato Exterior:', 'Último dato hall:', 'Último dato Policromos:')
    print(Cruce['Fecha/Tiempo'][len(Cruce['Fecha/Tiempo'])-1], Exterior['Fecha/Tiempo'][len(Exterior['Fecha/Tiempo'])-1],
          FicheroHall['Fecha/Tiempo'][len(FicheroHall['Fecha/Tiempo'])-1],Fecha_Tiempo[len(Tp)-1])
    return(Tp, Tp_suavizada, CO2, CO2_suavizado, Fecha_Tiempo)

Hay algunos meses donde los datos están desplazados 8 minutos, no merece la pena borrar datos para emparejar los tiempos porque la diferencia temporal seguiría siendo de 7 minutos, solo un minuto de diferencia.

# 0.2) Separar datos por semanas

### Para Cruce, exterior y hall:

In [3]:
def SepararPorSemanas(NombreFichero):
    """
    Introduciendo el nombre de un archivo excel de Altamira (NombreFichero), devuelve weekly_data, una lista de dataframes cuyos
    elementos corresponden cada uno a una semana del mes. Los dataframes de weekly_data tendrán las mismas columnas que el 
    archivo original.
    """
    df=pd.read_excel(NombreFichero)
    #Convertir la columna de datos temporales ('Fecha/Tiempo') a formato datetime:
    df['Fecha/Tiempo'] = pd.to_datetime(df['Fecha/Tiempo'])
    #Extraer el número de la semana de la columna de fechas:
    df['week'] = df['Fecha/Tiempo'].dt.week
    #Crear una lista vacía para guardar los datos (dataframe) de cada semana:
    weekly_data = []
    #Agrupar los datos con 'week', empezando por el lunes:
    grouped_df = df.groupby(pd.Grouper(key='Fecha/Tiempo', freq='W-SUN'))
    #Introducir los dataframes con los datos de una semana en la variable weekly_data:
    for week, data in grouped_df:
        weekly_data.append(data.reset_index(drop=True))
    return weekly_data 

### Separar por semanas policromos:

In [4]:
def separar_policromos_semanas(datosPolicromos):
    """
    Introduciendo como atributo datosPolicromos, convierte este en un dataframe y lo separa por semanas. 
    Devuelve weekly_data, una lista de dataframes. Cada dataframe contiene los datos proporcionados en datosPolicromos de una
    semana.
    
    Recordar que la estructura de datosPolicromos es:
    datosPolicromos = Tp, Tp_suavizada, CO2, CO2_suavizado, Fecha_Tiempo
    """
    
    #Pasar lista de listas a diccionario:
    diccionario_policromos_10_20={'Tp': datosPolicromos[0], 'Tp_suavizada': datosPolicromos[1], 'CO2': datosPolicromos[2],
                                  'CO2_suavizado': datosPolicromos[3], 'Fecha/Tiempo': datosPolicromos[4]}
    #Pasar el diccionario a dataframe:
    df = pd.DataFrame(data=diccionario_policromos_10_20)
    
    ##Separar por semanas:
    df['Fecha/Tiempo'] = pd.to_datetime(df['Fecha/Tiempo'])
    df['week'] = df['Fecha/Tiempo'].dt.week
    weekly_data = []
    grouped_df = df.groupby(pd.Grouper(key='Fecha/Tiempo', freq='W-SUN'))
    for week, data in grouped_df:
        weekly_data.append(data.reset_index(drop=True))
    return weekly_data 